In [1]:
import torch
import torch.nn as nn
from tqdm import tqdm
from nuscenes.nuscenes import NuScenes
from nuscenes.utils.splits import create_splits_scenes
from nuscenes.can_bus.can_bus_api import NuScenesCanBus

import sys
# sys.path.append('/mnt/cache/wangxiaodong/SDM/src')

In [2]:
DATAROOT = '/mnt/storage/user/wangxiaodong/nuscenes-all'

class frameDataset():
    def __init__(self, split='val'):
        self.split = split

        self.nusc = NuScenes(version='v1.0-trainval', dataroot=DATAROOT, verbose=True)

        self.splits = create_splits_scenes()

        self.samples_groups = self.get_samples(split)
        
    def __len__(self):
        return len(self.samples)
    
    def get_all_frames_from_scene(self, scene):
        # get all frames (keyframes, sweeps)
        first_sample_token = scene['first_sample_token']
        my_sample = self.nusc.get('sample', first_sample_token)
        sensor = "CAM_FRONT"
        cam_front_data = self.nusc.get('sample_data', my_sample['data'][sensor]) # first frame sensor token
        # frames = 0
        all_frames_dict = [] # len() -> frame number
        while True:
            all_frames_dict.append(cam_front_data)
            # filename = cam_front_data['filename']  # current-frame filename
            next_sample_data_token = cam_front_data['next']  # next-frame sensor token
            if not next_sample_data_token: # ''
                break
            cam_front_data = self.nusc.get('sample_data', next_sample_data_token)
            # frames += 1
        
        return all_frames_dict
    def get_samples(self, split='train'):
        selected_scenes = self.splits[split] # all scenes
        all_scenes = self.nusc.scene
        selected_scenes_meta = []
        for sce in all_scenes:
            if sce['name'] in selected_scenes:
                selected_scenes_meta.append(sce)
        
        samples_group_by_scene = {}
        for scene in selected_scenes_meta:
            scene_name = scene['name']
            samples_group_by_scene[scene_name] = self.get_all_frames_from_scene(scene)
        
        return samples_group_by_scene

In [12]:
dataset = frameDataset('train')
scenes_list = list(dataset.samples_groups.keys())
nusc_can = NuScenesCanBus(dataroot='/mnt/storage/user/wangxiaodong/nuscenes')

Loading NuScenes tables for version v1.0-trainval...
23 category,
8 attribute,
4 visibility,
64386 instance,
12 sensor,
10200 calibrated_sensor,
2631083 ego_pose,
68 log,
850 scene,
34149 sample,
2631083 sample_data,
1166187 sample_annotation,
4 map,
Done loading in 41.185 seconds.
Reverse indexing ...
Done reverse indexing in 7.6 seconds.


In [4]:
def get_sample_utime(token):
    sample_path = dataset.nusc.get('sample_data', token)['filename']
    sample_utime = int(sample_path.split('__')[-1].split('.')[0])
    return sample_utime

def get_sample_filename(token):
    sample_path = dataset.nusc.get('sample_data', token)['filename']
    return sample_path

def sort_samples(unsort_samples):
    samples_with_utime = []

    for item in unsort_samples:
        token = item['data']['CAM_FRONT']
        sample_utime = get_sample_utime(token)
        samples_with_utime.append((sample_utime, item))

    samples_with_utime.sort(key=lambda x: x[0])

    sorted_samples = [item for _, item in samples_with_utime]

    return sorted_samples

In [5]:
import bisect

def interpolate_speed(veh_speed, timestamp):
    if timestamp < veh_speed[0]['utime']:
        return veh_speed[0]['vehicle_speed']
    # 找到timestamp在veh_speed中的位置
    index = bisect.bisect_left([d['utime'] for d in veh_speed], timestamp)
    
    # 如果timestamp在veh_speed的最后一个元素之后，则返回最后一个元素的速度
    if index == len(veh_speed):
        return veh_speed[-1]['vehicle_speed']
    
    # 如果timestamp正好等于某个utime，则直接返回该utime对应的速度
    if veh_speed[index]['utime'] == timestamp:
        return veh_speed[index]['vehicle_speed']
    
    # 如果timestamp在两个utime之间，则进行插值
    prev_utime = veh_speed[index - 1]['utime']
    next_utime = veh_speed[index]['utime']
    prev_speed = veh_speed[index - 1]['vehicle_speed']
    next_speed = veh_speed[index]['vehicle_speed']
    
    # 进行线性插值
    interpolated_speed = prev_speed + ((next_speed - prev_speed) / (next_utime - prev_utime)) * (timestamp - prev_utime)
    
    return interpolated_speed

def find_nearest_speed(veh_speed, sorted_samples):
    result = []
    for sample in sorted_samples:
        timestamp = sample['timestamp']
        if timestamp > veh_speed[-1]['utime']:
            continue
        nearest_speed = interpolate_speed(veh_speed, timestamp)
        result.append(nearest_speed)
    return result

In [6]:
import bisect

def interpolate_angle(veh_angles, timestamp):
    if timestamp < veh_angles[0]['utime']:
        return veh_angles[0]['value']
    # 找到timestamp在veh_speed中的位置
    index = bisect.bisect_left([d['utime'] for d in veh_angles], timestamp)
    
    # 如果timestamp在veh_speed的最后一个元素之后，则返回最后一个元素的速度
    if index == len(veh_angles):
        return veh_angles[-1]['value']
    
    # 如果timestamp正好等于某个utime，则直接返回该utime对应的速度
    if veh_angles[index]['utime'] == timestamp:
        return veh_angles[index]['value']
    
    # 如果timestamp在两个utime之间，则进行插值
    prev_utime = veh_angles[index - 1]['utime']
    next_utime = veh_angles[index]['utime']
    prev_angle = veh_angles[index - 1]['value']
    next_angle = veh_angles[index]['value']
    
    # 进行线性插值
    interpolated_angle = prev_angle + ((next_angle - prev_angle) / (next_utime - prev_utime)) * (timestamp - prev_utime)
    
    return interpolated_angle

def find_nearest_angle(veh_angles, sorted_samples):
    result = []
    for sample in sorted_samples:
        timestamp = sample['timestamp']
        if timestamp > veh_angles[-1]['utime']:
            continue
        nearest_speed = interpolate_angle(veh_angles, timestamp)
        result.append(nearest_speed)
    return result

In [8]:
len(scenes_list)

150

In [11]:
# extract angle and speed for 40 frames in all scene
# val
meta_data = []
for sce in tqdm(scenes_list):
    unsort_samples = dataset.samples_groups[sce]
    # sorted_samples = sort_samples(unsort_samples)
    sorted_samples = unsort_samples
    veh_angle = nusc_can.get_messages(sce, 'steeranglefeedback')
    veh_speed = nusc_can.get_messages(sce, 'vehicle_monitor')

    angles = find_nearest_angle(veh_angle, sorted_samples)
    speeds = find_nearest_speed(veh_speed, sorted_samples)
    # print(f'scene: {sce}, len of angles: {len(angles)}, len of speeds: {len(speeds)}')
    if len(angles)!= len(speeds):
        print(f'scene: {sce}, len of angles: {len(angles)}, len of speeds: {len(speeds)}')
    
    # print(len(angles), len(speeds))
    
    filenames = []
    for sample in sorted_samples:
        # token = sample['data']['CAM_FRONT']
        # sample_filename = get_sample_filename(token)
        # filenames.append(sample_filename)
        filenames.append(sample['filename'])

    meta_data.append(
        {
            'scene': sce,
            'files': filenames,
            'angles': angles,
            'speeds': speeds
        }
    )
len(meta_data)
import json
with open('scene_action_file_allframes_val.json', 'w') as f:
    json.dump(meta_data, f)

  4%|▍         | 6/150 [00:00<00:02, 51.89it/s]

226 226
230 230
229 229
226 226
231 231
234 234
224 224
231 231
229 229
229 229
235 235


 12%|█▏        | 18/150 [00:00<00:02, 53.97it/s]

233 233
225 225
229 229
233 233
227 227
228 228
231 231
228 228
224 224
229 229
230 230
232 232


 20%|██        | 30/150 [00:00<00:02, 54.05it/s]

...

233 233
237 237
234 234
238 238
231 231
235 235
234 234
236 236
233 233
232 232
235 235
235 235


 96%|█████████▌| 144/150 [00:02<00:00, 52.34it/s]

235 235
238 238
239 239
236 236
234 234
234 234
233 233
235 235
229 229
236 236
232 232


100%|██████████| 150/150 [00:02<00:00, 52.92it/s]


236 236
232 232
229 229


In [13]:
# extract angle and speed for 40 frames in all scene
# train
meta_data = []
for sce in scenes_list:
    unsort_samples = dataset.samples_groups[sce]
    # sorted_samples = sort_samples(unsort_samples)
    sorted_samples = unsort_samples # already sorted by -> next
    try:
        veh_angle = nusc_can.get_messages(sce, 'steeranglefeedback')
        veh_speed = nusc_can.get_messages(sce, 'vehicle_monitor')
    except:
        print(f'{sce} does not have any CAN bus data!')
        continue
    if len(veh_angle)==0 or len(veh_speed)==0:
        print(f'{sce} have zero CAN bus data!')
        continue

    angles = find_nearest_angle(veh_angle, sorted_samples)
    speeds = find_nearest_speed(veh_speed, sorted_samples)
    # print(f'scene: {sce}, len of angles: {len(angles)}, len of speeds: {len(speeds)}')
    if len(angles)!= len(speeds):
        print(f'scene: {sce}, len of angles: {len(angles)}, len of speeds: {len(speeds)}')
    
    filenames = []
    for sample in sorted_samples:
        # token = sample['data']['CAM_FRONT']
        # sample_filename = get_sample_filename(token)
        # filenames.append(sample_filename)
        filenames.append(sample['filename'])

    meta_data.append(
        {
            'scene': sce,
            'files': filenames,
            'angles': angles,
            'speeds': speeds
        }
    )
len(meta_data)
import json
with open('scene_action_file_allframes_train.json', 'w') as f:
    json.dump(meta_data, f)

scene-0161 does not have any CAN bus data!
scene-0162 does not have any CAN bus data!
scene-0163 does not have any CAN bus data!
scene-0164 does not have any CAN bus data!
scene-0165 does not have any CAN bus data!
scene-0166 does not have any CAN bus data!
scene-0167 does not have any CAN bus data!
scene-0168 does not have any CAN bus data!
scene-0170 does not have any CAN bus data!
scene-0171 does not have any CAN bus data!
scene-0172 does not have any CAN bus data!
scene-0173 does not have any CAN bus data!
scene-0174 does not have any CAN bus data!
scene-0175 does not have any CAN bus data!
scene-0176 does not have any CAN bus data!


/mnt/storage/user/wangxiaodong/.local/lib/python3.11/site-packages/nuscenes/can_bus/can_bus_api.py:236: UserWarning: Warning: %s does not have any vehicle_monitor messages!
  warnings.warn('Warning: %s does not have any vehicle_monitor messages!')


scene-0419 have zero CAN bus data!


In [13]:
len(meta_data)

684